In [1]:
from github import Github, RateLimitExceededException
import pandas as pd
import os
from datetime import datetime

from repository import Repository

In [2]:
# Read Token
access_token = open('GITHUB_TOKEN.txt', 'r').read()
user = Github(access_token)
user.per_page = 100

In [3]:
df = pd.read_json ('./data/python_best.json')

In [4]:
def check_rate_limit():
    rate_limit = user.get_rate_limit()
    remaining = rate_limit.search.remaining
    
    if remaining > 0:
        return
    
    reset_time = rate_limit.search.reset
    now = datetime.utcnow()

    if now < reset_time:
        sleep_time = (reset_time - now).total_seconds()
        print('=> Waiting {} seconds for rate limit reset.'.format(sleep_time))
        time.sleep(sleep_time)

In [ ]:
for i in range (0, 50):
    print("================ {} ================".format(i))
    try:
        r = user.get_repo(int(df['id'][i]))
    except RateLimitExceededException:
        check_rate_limit()
        continue
        
    repo = Repository(r, './repos/', './analysisPCC.json')
    repo.download()
    repo.analyse()
    repo.analyseComplexity()
    repo.dump()
    repo.delete()

================ 0 ================
Cloned Repository:  youtube-dl
Analysed Repository:  youtube-dl
Analysed Complexity of Repository:  youtube-dl
Dumped data from Repository: youtube-dl (into file: ./analysisPCC.json)
Deleted Repository:  youtube-dl
================ 1 ================
Cloned Repository:  12306
Analysed Repository:  12306
Analysed Complexity of Repository:  12306
Dumped data from Repository: 12306 (into file: ./analysisPCC.json)
Deleted Repository:  12306
================ 2 ================
Cloned Repository:  incubator-mxnet
Analysed Repository:  incubator-mxnet
Analysed Complexity of Repository:  incubator-mxnet
Dumped data from Repository: incubator-mxnet (into file: ./analysisPCC.json)
Deleted Repository:  incubator-mxnet
================ 3 ================
Cloned Repository:  sqlmap
Analysed Repository:  sqlmap
Analysed Complexity of Repository:  sqlmap
Dumped data from Repository: sqlmap (into file: ./analysisPCC.json)
Deleted Repository:  sqlmap
================

cannot read repos/ipython/IPython/core/tests/nonascii.py using encoding utf-8: 'utf-8' codec can't decode byte 0xb1 in position 81: invalid start byte


Analysed Repository:  ipython
Analysed Complexity of Repository:  ipython
Dumped data from Repository: ipython (into file: ./analysisPCC.json)
Deleted Repository:  ipython
================ 6 ================
Cloned Repository:  kivy
Analysed Repository:  kivy
Analysed Complexity of Repository:  kivy
Dumped data from Repository: kivy (into file: ./analysisPCC.json)
Deleted Repository:  kivy
================ 7 ================
Cloned Repository:  gensim
Analysed Repository:  gensim
Analysed Complexity of Repository:  gensim
Dumped data from Repository: gensim (into file: ./analysisPCC.json)
Deleted Repository:  gensim
================ 8 ================
Cloned Repository:  FastPhotoStyle
Analysed Repository:  FastPhotoStyle
Analysed Complexity of Repository:  FastPhotoStyle
Dumped data from Repository: FastPhotoStyle (into file: ./analysisPCC.json)
Deleted Repository:  FastPhotoStyle
================ 9 ================
Cloned Repository:  InstaPy
Analysed Repository:  InstaPy
Analysed Co